In [43]:
import numpy as np
import pandas as pd
import yfinance as yf
from os.path import exists
print("done importing")

done importing


In [44]:
# some helpful definitions
stocks = { # excludes SPY
    'BHP', 'NPCPF', 'VITFF', 
    'GOOG', 'ROKU', 'OPRA', 
    'LVMUY', 'EXPE', 'TUP', 
    'JPM', 'MET', 'ARES', 
    'PLD', 'DITTF', 'INN', 
    'PG', 'AJINY', 'DFIHY', 
    'UNH', 'LH', 'NVTA', 
    'NEE', 'TKGSY', 'AZRE', 
    'XOM', 'VVV', 'AROC', 
    'UPS', 'CMI', 'ROCK', 
    'AAPL', 'CAJ', 'APPN'}

directories = {
    'risk_data',
    'sector_data',
    'Mean Reversion'
    'stocks_equityreturn_operatingmodel'
}

risk_dict = {} # maps stock ticker string to dataframe of its factors
equity_operatingmodel_dict = {}
mean_reversion_dict = {}
size_dict = {}
merged_dict = {}

def get_risk_file(stock):
    return 'risk_data\\' + stock + '_risk_factors.csv'

def get_equity_operatingmodel_file(stock):
    return 'stocks_equityreturn_operatingmodel\\' + stock + '_CSV.csv'

def get_mean_reversion_file(stock):
    return 'Mean Reversion\\' + stock + '_CSV'

def get_size_file(stock):
    return 'Mean Reversion\\' + stock + '_Size'

In [45]:
# importing files
success = True # set to false if file is missing or doesn't load

for stock in stocks:
    risk_file = get_risk_file(stock)
    equity_operatingmodel_file = get_equity_operatingmodel_file(stock)
    mean_reversion_file = get_mean_reversion_file(stock)
    size_file = get_size_file(stock)

    if(exists(risk_file) == False):
        success = False
        print (risk_file + ' not loaded')
    else:
        risk_dict[stock] = pd.read_csv(risk_file, index_col='Date')

    if(exists(equity_operatingmodel_file) == False):
        success = False
        print (equity_operatingmodel_file + ' not loaded')
    else:
        equity_operatingmodel_dict[stock] = pd.read_csv(equity_operatingmodel_file, index_col='Date')

    if(exists(mean_reversion_file) == False):
        success = False
        print (mean_reversion_file + ' not loaded')
    else:
        mean_reversion_dict[stock] = pd.read_csv(mean_reversion_file, index_col='Date')

    if(exists(size_file) == False):
        success = False
        print (size_file + ' not loaded')
    else:
        size_dict[stock] = pd.read_csv(size_file, index_col='Date')
        

sector_file = 'sector_data\\sector_classification.csv'
if(exists(sector_file) == False):
    success = False
    print (sector_file + ' not loaded')
else:
    sector_data = pd.read_csv(sector_file)

if success == True:
    print('all files loaded')



Mean Reversion\DFIHY_Size not loaded
Mean Reversion\VVV_Size not loaded
Mean Reversion\BHP_Size not loaded
Mean Reversion\LVMUY_Size not loaded
Mean Reversion\AJINY_CSV not loaded
Mean Reversion\AJINY_Size not loaded
Mean Reversion\VITFF_Size not loaded
Mean Reversion\CAJ_CSV not loaded
Mean Reversion\CAJ_Size not loaded
Mean Reversion\PG_CSV not loaded
Mean Reversion\PG_Size not loaded
Mean Reversion\APPN_Size not loaded
Mean Reversion\ROKU_Size not loaded
Mean Reversion\UPS_CSV not loaded
Mean Reversion\UPS_Size not loaded


In [46]:
# aligning dataset formatting

drop_columns = ['Unnamed: 0']

# changing all data to be datetime-indexed, and dropping unnecessary columns
for stock in stocks:
    risk_dict[stock].index = pd.to_datetime(risk_dict[stock].index) # converting Date column to DateTime format
    equity_operatingmodel_dict[stock].index = pd.to_datetime(equity_operatingmodel_dict[stock].index) 
    if(exists(get_mean_reversion_file(stock)) == True):
        mean_reversion_dict[stock].index = pd.to_datetime(mean_reversion_dict[stock].index) 
        mean_reversion_dict[stock].drop(drop_columns, inplace=True, axis=1)
    if(exists(get_size_file(stock)) == True):
        size_dict[stock].index = pd.to_datetime(size_dict[stock].index) 
        size_dict[stock].drop(drop_columns, inplace=True, axis=1)

# changing mean reversion and size to be within 2015-01-01 and 2021-01-01
start = '2015-01-01'
end = '2021-01-01' # last trading day in 2020
for stock in stocks:
    if(exists(get_mean_reversion_file(stock)) == True):
        mean_reversion_dict[stock] = mean_reversion_dict[stock].truncate(before=start, after=end)
    if(exists(get_size_file(stock)) == True):
        size_dict[stock] = size_dict[stock].truncate(before=start, after=end)

    
# changing risk data and equity/operating model data to have intersection of dates
common_index = risk_dict['AAPL'].index.intersection(equity_operatingmodel_dict['AAPL'].index) # should work with any ticker
for stock in stocks:
    equity_operatingmodel_dict[stock] = equity_operatingmodel_dict[stock].loc[common_index]



In [47]:
# joining the dataframes
for stock in stocks:
    datasets = [risk_dict[stock], equity_operatingmodel_dict[stock]]
    if(exists(get_mean_reversion_file(stock)) == True):
        datasets.append(mean_reversion_dict[stock])
    if(exists(get_size_file(stock)) == True):
        datasets.append(size_dict[stock])
    merged_dict[stock] = pd.concat(datasets, axis=1, ignore_index = False)

# adding null columns if there is no size or no csv file
mean_reversion_columns = ['STMR', 'momentum', 'ltmr']
size_columns = ['Size', 'P/B Ratio']
for stock in stocks:
    if(exists(get_mean_reversion_file(stock)) == False):
        for col in mean_reversion_columns:
            merged_dict[stock][col] = np.nan
    if(exists(get_size_file(stock)) == False):
        for col in size_columns:
            merged_dict[stock][col] = np.nan

In [48]:
# adding sector classification
sector_columns = {'Unnamed: 0':'Ticker', 'Sector':'Sector', 'Sector Index':'Sector Index'}
sector_data = sector_data.rename(sector_columns, axis=1)
sector_data.index = sector_data['Ticker']

for stock in stocks:
    merged_dict[stock]['Sector Index'] = sector_data['Sector Index'].loc[stock]

In [49]:
# generating csv and storing them in \merged_data
for stock in stocks:
    merged_dict[stock].to_csv('merged_data\\' + stock + 'data.csv')
print('csvs generated')

csvs generated
